In [7]:
# Import libraries adn load dependencies
import torch
import numpy as np
import pandas as pd

# Models:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification # for german
from pattern.en import sentiment # for english
import nltk # for english
from nltk.sentiment import SentimentIntensityAnalyzer # for english
nltk.download('vader_lexicon') # for english
from pysentimiento import create_analyzer # for spanish

# Accuracy
# from utils import evaluate_performance #ACTION: uncomment and put utils function away
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
def evaluate_performance(df, sentiment_column, label_column):
    # Calculate the accuracy
    accuracy = accuracy_score(df[label_column], df[sentiment_column])

    # Find unique predicted sentiment labels
    unique_predicted = df[sentiment_column].unique()

    # Assign true and predicted labels
    true_labels = df[label_column]
    predicted_labels = df[sentiment_column]

    # Create the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Convert the confusion matrix to a DataFrame for better visualization
    labels = np.unique(np.concatenate((true_labels, predicted_labels)))
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)


    # Generate the classification report
    report = classification_report(true_labels, predicted_labels)

    return accuracy, unique_predicted, cm_df, report

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joana\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# More sentiment models and their accuracy per language

## Intro
TODO

full text
extracted sentence

full condensed
extracted sentence

### Load labeled data: d1.1
Explanation why we used d1.1 and that is already with extracted sentence 

In [8]:
# Load labeled CSV files into a DataFrame
df_de_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-de_clean_1-1.csv', sep=';')
df_en_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-en_clean_1-1_not101010.csv')
df_es_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-es_clean_1-1.csv', sep=';')

### Load labeled data: data condensed for english
Explanation why we used data condensed for english

In [9]:
# Load labeled CSV file into a DataFrame
df_en_con = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-en_clean_con_sen.csv')

## German
For the german data we used one additional model to the multilingual only, because then we focused on clustering.

### Model: AutoTokenizer and AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")
The model used was trained on 1.834 million German-language samples sepcifically for sentiment classification.
https://huggingface.co/oliverguhr/german-sentiment-bert

In [10]:
def german_sentiment_model(df):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

    # Create an empty list to store the sentiment scores
    sentiment_scores = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Tokenize the input text
        tokens = tokenizer.encode_plus(text, padding="max_length", truncation=True, max_length=128,
                                       return_tensors="pt")

        # Perform the sentiment analysis
        with torch.no_grad():
            logits = model(**tokens)[0]

        # Convert logits to predicted label (positive/negative)
        predicted_label = torch.argmax(logits, dim=1).item()
        sentiment = "positive" if predicted_label == 1 else "negativ"

        # Append the sentiment score to the list
        sentiment_scores.append(sentiment)

    # Add the sentiment scores as a new column in the DataFrame
    df['sentiment_bert'] = sentiment_scores

    return df


# Perform sentiment analysis on the DataFrame
df_de_1 = german_sentiment_model(df_de_1)

# Print the updated DataFrame
print(df_de_1)



#ACTION: NOTE FOR JOANA: check whether this makes a new dataframe instead of appending a column. If so: Change & check what 

                                                  data    player language  \
0    trainer alonso vor den mitgereisten fans in mo...  palacios       de   
1    zudem ist die konkurrenzsituation auf der dopp...  palacios       de   
2    wie auch palacios sah der defensive mittelfeld...  palacios       de   
3    ,,er ist eine option", erklart alonso, der im ...  palacios       de   
4       allerdings waren in andrich und dem argenti...  palacios       de   
..                                                 ...       ...      ...   
434  vor allem monacos krepin diatta und bayerlever...  hincapie       de   
435     vor allem monacos krepin diatta und bayerle...  hincapie       de   
436  in der funften minute der nachspielzeit sah ba...  hincapie       de   
437  une febrilite deconcertante venant du bayerlev...  hincapie       de   
438  sekunden waren gespielt, als andrich nach here...  hincapie       de   

              publishedAt    Label sentiment_bert  
0    2023-02-24T09:33:3

#### Evaluate model performance for german bert model
TODO and todo code

In [11]:
# Drop rows where 'Label' is NaN or empty
df_de_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_de_1_pos_neg = df_de_1[df_de_1['Label'] != 'neutral']

In [14]:
print('Performance evaluation for oliverguhr/german-sentiment-bert')
accuracy_de, unique_predicted_de, confusion_matrix_de, classification_report_de = evaluate_performance(df_de_1_pos_neg, 'sentiment_bert', 'Label')

print('Accuracy: ', accuracy_de,' Unique labels: ', unique_predicted_de)

print('Confusion matrix: ')
print(confusion_matrix_de)
print('Classification report: ')
print(classification_report_de)

Performance evaluation for oliverguhr/german-sentiment-bert
Accuracy:  0.5  Unique labels:  ['negativ']
Confusion matrix: 
         negativ  positiv
negativ       10        0
positiv       10        0
Classification report: 
              precision    recall  f1-score   support

     negativ       0.50      1.00      0.67        10
     positiv       0.00      0.00      0.00        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344

## English - d1.1
TODO

### Why we used the pipeline for the english and spanish models
TODO


### Model 1: sentiment-analysis from bert-base-uncased
TODO
https://huggingface.co/bert-base-uncased

In [16]:
# Initiate model
sentiment_classifier_en = pipeline('sentiment-analysis', model='bert-base-uncased')

# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_bert"
df_en_1['sentiment_bert'] = df_en_1['data'].apply(lambda x: sentiment_classifier_en(x)[0]['score'])

# Print the updated dataframe
df_en_1.head()

,data,player,language,publishedAt,Label,sentiment_bert
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN,0.566346
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN,0.581037
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.619449
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.622141
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.622096


### Model 2: Sentiment Intensity Analyzer from nltk

In [20]:
# Create an instance of the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment polarity
def get_sentiment(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']


# Apply sentiment analysis to the "data" column and store the sentiment in a new column "sentiment_nltk"
df_en_1['sentiment_nltk'] = df_en_1['data'].apply(get_sentiment)

# Print the updated dataframe
df_en_1.head()


,data,player,language,publishedAt,Label,sentiment_bert,sentiment_pattern,sentiment_nltk
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN,0.566346,"(0.0, 0.0)",0.0000
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN,0.581037,"(0.20833333333333334, 0.3)",-0.0516
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.619449,"(0.1, 0.2)",0.2263
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.622141,"(0.0, 0.0)",0.0000
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.622096,"(0.2, 0.30000000000000004)",0.0000


#### Evaluate model performance for all english models
TODO 

In [21]:
# Drop rows where 'Label' is NaN or empty
df_en_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_en_1_pos_neg = df_en_1[df_en_1['Label'] != 'neutral']

In [23]:
print('Performance evaluation for bert-base-uncased with two-dimensional labels (positive/negative)')

# Transform score into two-dimensional label for Performance evaluation
sentiment_2_labels = []
for score in df_en_1_pos_neg['sentiment_bert']:        
    # Convert sentiment score to two-dimensional label (positive/negative)
    sentiment_2_label = "positiv" if score > 0 else "negativ"
    sentiment_2_labels.append(sentiment_2_label)
df_en_1_pos_neg['sentiment_2_label_bert'] = sentiment_2_labels

# Evaluate the performance of the model
accuracy_en_bert, unique_predicted_en_bert, confusion_matrix_en_bert, classification_report_en_bert = evaluate_performance(df_en_1_pos_neg, 'sentiment_2_label_bert', 'Label')

# Print the evaluation results
print('Accuracy: ', accuracy_en_bert, ' Unique labels: ', unique_predicted_en_bert)
print('Confusion matrix: ')
print(confusion_matrix_en_bert)
print('Classification report: ')
print(classification_report_en_bert)


Performance evaluation for bert-base-uncased with two-dimensional labels (positive/negative) 
Accuracy:  0.7647058823529411  Unique labels:  ['positiv']
Confusion matrix: 
         negativ  positiv
negativ        0        4
positiv        0       13
Classification report: 
              precision    recall  f1-score   support

     negativ       0.00      0.00      0.00         4
     positiv       0.76      1.00      0.87        13

    accuracy                           0.76        17
   macro avg       0.38      0.50      0.43        17
weighted avg       0.58      0.76      0.66        17



C:\Users\joana\AppData\Local\Temp\ipykernel_28752\2779216896.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_1_pos_neg['sentiment_2_label_bert'] = sentiment_2_labels
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classificatio

In [32]:
print('Performance evaluation for nltk')

# Transform score into two-dimensional label for Performance evaluation
sentiment_2_labels = []
for score in df_en_1_pos_neg['sentiment_nltk']:        
    # Convert sentiment score to two-dimensional label (positive/negative)
    sentiment_2_label = "positiv" if score > 0 else "negativ"
    sentiment_2_labels.append(sentiment_2_label)
df_en_1_pos_neg['sentiment_2_label_nltk'] = sentiment_2_labels

accuracy_en_nltk, unique_predicted_en_nltk, confusion_matrix_en_nltk, classification_report_en_nltk = evaluate_performance(df_en_1_pos_neg, 'sentiment_2_label_nltk', 'Label')

print('Accuracy: ', accuracy_en_nltk, ' Unique labels: ', unique_predicted_en_nltk)

print('Confusion matrix: ')
print(confusion_matrix_en_nltk)
print('Classification report: ')
print(classification_report_en_nltk)

Performance evaluation for nltk
Accuracy:  0.6470588235294118  Unique labels:  ['positiv' 'negativ']
Confusion matrix: 
         negativ  positiv
negativ        3        1
positiv        5        8
Classification report: 
              precision    recall  f1-score   support

     negativ       0.38      0.75      0.50         4
     positiv       0.89      0.62      0.73        13

    accuracy                           0.65        17
   macro avg       0.63      0.68      0.61        17
weighted avg       0.77      0.65      0.67        17



C:\Users\joana\AppData\Local\Temp\ipykernel_28752\695854752.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_1_pos_neg['sentiment_2_label_nltk'] = sentiment_2_labels


## Spanish
TODO?

### Model 1: sentiment-analysis from spanish bert: beto-sentiment-analysis
TODO
https://huggingface.co/finiteautomata/beto-sentiment-analysis
Although this model was trained on tweets, not on news data, it is a popular model to use for sentiment analysis of spanish data.
The base model is BETO, a BERT model trained on spanish data (for further information: https://github.com/dccuchile/beto).

In [ ]:
sentiment_classifier_es_beto = pipeline('sentiment-analysis', model='finiteautomata/beto-sentiment-analysis')

In [ ]:
# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_beto"
df_es_1['sentiment_beto'] = df_es_1['data'].apply(lambda x: sentiment_classifier_es_beto(x)[0]['score'])

# Print the updated dataframe
df_es_1.head()

### Model 2: sentiment-analysis from another spanish bert: bert-base-spanish-wwm-uncased
TODO

In [ ]:
sentiment_classifier_es_bert = pipeline('sentiment-analysis', model='dccuchile/bert-base-spanish-wwm-uncased')

In [ ]:
# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_bert"
df_es_1['sentiment_bert'] = df_es_1['data'].apply(lambda x: sentiment_classifier_es_bert(x)[0]['score'])

# Print the updated dataframe
df_es_1.head()

#### Evaluate model performance for all spanish models
TODO 

In [ ]:
# Drop rows where 'Label' is NaN or empty
df_es_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_es_1_pos_neg = df_es_1[df_es_1['Label'] != 'neutral']

In [ ]:
print('Performance evaluation for beto-sentiment-analysis')

# Calculate the accuracy
accuracy_es_beto = (df_es_1_pos_neg['sentiment_beto'] == df_es_1_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for beto: {:.2f}%".format(accuracy_es_beto))



# Print unique predicted sentiment labels
print('Unique labels for beto:')
print(df_es_1_pos_neg['sentiment_beto'].unique())


# # Assign true and predicted labels
true_labels = df_es_1_pos_neg['Label']
predicted_labels = df_es_1_pos_neg['sentiment_beto']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_es_beto = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for beto:")
print(cm_df_es_beto)



# Generate the classification report
report_es_beto = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for beto:")
print(report_es_beto)

In [ ]:
print('Performance evaluation for bert-base-spanish-wwm-uncased')


# Calculate the accuracy
accuracy_es_bert = (df_es_1_pos_neg['sentiment_bert'] == df_es_1_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for bert: {:.2f}%".format(accuracy_es_bert))



# Print unique predicted sentiment labels
print('Unique labels for bert:')
print(df_es_1_pos_neg['sentiment_bert'].unique())


# # Assign true and predicted labels
true_labels = df_es_1_pos_neg['Label']
predicted_labels = df_es_1_pos_neg['sentiment_bert']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_es_bert = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for bert:")
print(cm_df_es_bert)



# Generate the classification report
report_es_bert = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for beto:")
print(report_es_bert)

## English: data condensed
TODO

### Model 1: sentiment-analysis from bert-base-uncased
TODO
https://huggingface.co/bert-base-uncased

In [ ]:
# Model loaded above: sentiment_classifier_en = pipeline('sentiment-analysis', model='bert-base-uncased')

# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_bert"
df_en_con['sentiment_bert'] = df_en_con['data'].apply(lambda x: sentiment_classifier_en(x)[0]['score'])

# Print the updated dataframe
df_en_con.head()

#### Evaluate model performance for english condensed model

In [ ]:
# Drop rows where 'Label' is NaN or empty
df_en_con.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_en_con_pos_neg = df_en_con[df_en_1['Label'] != 'neutral']

In [ ]:
print('Performance evaluation for bert-base-uncased on english condensed')

# Calculate the accuracy
accuracy_en_con_bert = (df_en_con_pos_neg['sentiment_bert'] == df_en_con_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for bert: {:.2f}%".format(accuracy_en_con_bert))



# Print unique predicted sentiment labels
print('Unique labels for bert:')
print(df_en_con_pos_neg['sentiment_bert'].unique())


# # Assign true and predicted labels
true_labels = df_en_con_pos_neg['Label']
predicted_labels = df_en_con_pos_neg['sentiment_bert']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_en_con_bert = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for bert:")
print(cm_df_en_con_bert)



# Generate the classification report
report_en_con_bert = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for bert:")
print(report_en_con_bert)